# MNIST 손글씨 - CNN v1
- 2 Conv2D layers
- 1 Fully Connected Network layer
- 1 Output layer

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

- 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
# X data: 0~1 사이의 값, 3차원 --> 4차원
X_train = X_train.reshape(-1,28,28,1) / 255.
X_test = X_test.reshape(-1,28,28,1) / 255.

In [4]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

- 모델 정의, 설정, 학습, 평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([ 
    Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 512)               18874880  
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model_path = 'best-mnist-cnn-v1.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [9]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2, epochs=100, batch_size=200,
    callbacks=[checkpoint, early_stop]
)

Epoch 1/100
239/240 [============================>.] - ETA: 0s - loss: 0.1524 - accuracy: 0.9530
Epoch 00001: val_loss improved from inf to 0.05610, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 21s 22ms/step - loss: 0.1520 - accuracy: 0.9531 - val_loss: 0.0561 - val_accuracy: 0.9837
Epoch 2/100
240/240 [==============================] - ETA: 0s - loss: 0.0354 - accuracy: 0.9891
Epoch 00002: val_loss improved from 0.05610 to 0.05195, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 5s 21ms/step - loss: 0.0354 - accuracy: 0.9891 - val_loss: 0.0520 - val_accuracy: 0.9858
Epoch 3/100
239/240 [============================>.] - ETA: 0s - loss: 0.0174 - accuracy: 0.9948
Epoch 00003: val_loss improved from 0.05195 to 0.04574, saving model to best-mnist-cnn-v1.h5
240/240 [==============================] - 5s 21ms/step - loss: 0.0173 - accuracy: 0.9948 - val_loss: 0.0457 - val_accuracy: 0.9869
Epoch 4/100
238/240 [================

In [10]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0417 - accuracy: 0.9879


[0.04165932536125183, 0.9879000186920166]